In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator as data_augment

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade_left = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')



In [ ]:
import cv2
import numpy as np

def detect_face_and_eyes(frame, target_size=(224, 224)):
    normalized_frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    frame_u8 = np.uint8(normalized_frame)
    
    gray = cv2.cvtColor(frame_u8, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes_left = eye_cascade_left.detectMultiScale(roi_gray)

        if len(eyes_left) > 0:
            left_eye = roi_gray[eyes_left[0][1]:eyes_left[0][1]+eyes_left[0][3], eyes_left[0][0]:eyes_left[0][0]+eyes_left[0][2]]
            face = roi_gray

            # Resize to target size
            left_eye = cv2.resize(left_eye, target_size)
            face = cv2.resize(face, target_size)

            # Expand dimensions to fit target shape (224, 224, 1)
            left_eye = np.expand_dims(left_eye, axis=2)
            face = np.expand_dims(face, axis=2)

            return face, left_eye

    return None, None


In [ ]:
# Function to process frames and labels with face detection
def process_frames_with_eyes(frames):
    left_eye_frames = []
    face_frames = []
    
    for frame in frames:
    
        # Detect face and eyes
        face_regions, left_eye_regions = detect_face_and_eyes(frame)

        for face, left_eye in zip(face_regions, left_eye_regions):
            # Resize left eye region
            
            if (left_eye is not None and face is not None):
                # Normalize pixel values
                left_eye = left_eye / 255.0
                face = face / 255.0

                left_eye_frames.append(left_eye)
                face_frames.append(face)

    return face_frames, left_eye_frames


In [ ]:
# Process frames with face detection
left_eye_frames, right_eye_frames = process_frames_with_eyes(features)

In [ ]:
# Assuming left_eye_frames and right_eye_frames are lists of left and right eye frames
left_eye_frames = np.array(left_eye_filtered)
labels_filtered = np.array(labels_filtered)

# Convert labels to numeric format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_filtered)
labels_encoded = np.array(labels_encoded)

In [ ]:
np.savenpz('eye_data.npz', left_eye_frames = left_eye_frames, right_eye_frames  = right_eye_frames, labels = labels_encoded)

In [ ]:
#data augmetation 
data_generate_training = data_augment (rescale=1./255, 
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              fill_mode = "nearest",
                              horizontal_flip = True,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              validation_split = 0.15)

data_generate_test = data_augment(rescale = 1./255)


In [ ]:
#data preprocessing and augmentation
traind = data_generate_training.flow_from_directory("Training Data",
                                          target_size = (224, 224),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "training")

testd = data_generate_training.flow_from_directory("Training Data"/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset4(DDD4",
                                          target_size = (227, 227),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "validation")